# template から HTML と JS を生成する


In [3]:
import os
from pathlib import Path
import json
import gzip
from collections import defaultdict

In [4]:
# Move up to top directory

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI
/mnt/e/Research/TSUMUGI


# 古いファイルの消去


In [ ]:
%%bash

rm -rf TSUMUGI/data
rm -rf TSUMUGI/web

mkdir -p TSUMUGI/data/phenotype
mkdir -p TSUMUGI/data/genesymbol
mkdir -p TSUMUGI/web/phenotype
mkdir -p TSUMUGI/web/genesymbol


## index.html の描画に必要なファイルを生成する


In [5]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    if not Path("data", "network", "mp_term_name", f"{mp_term}.json.gz").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    mp_terms[mp_term_name_space] = mp_term

In [6]:
print(list(mp_terms)[:3])
print(len(mp_terms))  # 515

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
517


In [7]:
json.dump(mp_terms, open("data/overlap/available_mp_terms.json", "w"), indent=2)

In [8]:
%%bash

cp data/overlap/available_mp_terms.json TSUMUGI/data/available_mp_terms.json
cp data/overlap/available_gene_symbols.txt TSUMUGI/data/available_gene_symbols.txt
cp data/annotation/symbol_mgiid.json TSUMUGI/network/data/marker_symbol_accession_id.json


## index.html を生成


In [ ]:
%%bash

grep -v "REMOVE_THIS_LINE" TSUMUGI/template_index.html > TSUMUGI/TSUMUGI.html

# JSON のコピー


In [10]:
%%bash

rm -rf TSUMUGI/network/phenotype/data
rm -rf TSUMUGI/network/genesymbol/data

mkdir -p TSUMUGI/network/phenotype/data
mkdir -p TSUMUGI/network/genesymbol/data

cp -r data/network/mp_term_name/* TSUMUGI/network/phenotype/data
cp -r data/network/gene_symbol/* TSUMUGI/network/genesymbol/data

# 1m30s

In [11]:
path_files = Path("TSUMUGI", "network", "phenotype", "data").glob("*.json.gz")
binary_phenotypes = []
for path_file in path_files:
    mp_term_name = path_file.name.replace(".json.gz", " ")
    with gzip.open(path_file, "rt") as f:
        json_data: list[dict] = json.load(f)

    count_node_color = defaultdict(int)
    for data in json_data:
        if "node_color" not in data["data"]:
            continue
        count_node_color[data["data"]["node_color"]] += 1
    if len(count_node_color) == 1 and 1 in count_node_color:
        binary_phenotypes.append(mp_term_name)

print(len(binary_phenotypes))
print(binary_phenotypes[:3])
Path("data", "annotation", "binary_phenotypes.txt").write_text("\n".join(binary_phenotypes))

154
['abnormal_abdominal_wall_morphology ', 'abnormal_adrenal_gland_morphology ', 'abnormal_allantois_morphology ']


3761

# Template から HTML と JS を生成


In [12]:
%%bash

########################################
# Phenotype
########################################

mkdir -p TSUMUGI/network/phenotype/js

find data/mp_term_name -type f |
    sed "s|data/mp_term_name/||" |
    sed "s|.csv$||" |
    while read -r mp_term_name_underscore; do
        mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
        impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/annotation/mptermname_phenotypeurl.tsv)
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
        # --------------------------
        # HTML
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのフィルターを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/network/template/template_network_phenotype.html |
            sed '/<!-- REMOVE FROM THIS LINE IF BINARY -->/,/<!-- REMOVE TO THIS LINE -->/d' |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/network/phenotype/"$mp_term_name_underscore".html
        else
            cat TSUMUGI/network/template/template_network_phenotype.html |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/network/phenotype/"$mp_term_name_underscore".html
        fi

        # --------------------------
        # Javascript
        # --------------------------
        cat TSUMUGI/network/template/template_network_phenotype.js |
        grep -v "REMOVE_THIS_LINE" |
        sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/network/phenotype/js/"$mp_term_name_underscore".js
    done
# 24.4s

In [13]:
%%bash

########################################
# Gene Symbol
########################################

mkdir -p TSUMUGI/network/genesymbol/js

cat data/overlap/available_gene_symbols.txt |
    while read gene_symbol; do
        mgi_id=$(awk -F "\t" -v key="MGI:1918912" '$1 == key {print $2}' data/annotation/symbol_mgiid.tsv)
        impc_url="https://www.mousephenotype.org/data/genes/"${mgi_id}
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # HTML
        cat TSUMUGI/network/template/template_network_genesymbol.html |
        grep -v "REMOVE_THIS_LINE" |
        sed "s|XXX_impc_url|${escaped_url}|g" |
        sed "s|XXX_genesymbol|${gene_symbol}|g" > TSUMUGI/network/genesymbol/"$gene_symbol".html

        # Javascript
        cat TSUMUGI/network/template/template_network_genesymbol.js |
        grep -v "REMOVE_THIS_LINE" |
        sed "s|XXX_genesymbol|${gene_symbol}|g" > TSUMUGI/network/genesymbol/js/"$gene_symbol".js
    done

# 5 min

## テスト用のデータを test-tsumugi にコピー


In [22]:
%%bash

rm -rf test-tsumugi
mkdir -p test-tsumugi/network/
echo "*" > test-tsumugi/.gitignore

mkdir -p test-tsumugi/data
mkdir -p test-tsumugi/network/phenotype
mkdir -p test-tsumugi/network/genesymbol
(cd test-tsumugi/network/phenotype && mkdir -p data js)
(cd test-tsumugi/network/genesymbol && mkdir -p data js)

In [23]:
%%bash

cp -r TSUMUGI/index.html test-tsumugi/
cp -r TSUMUGI/css test-tsumugi/
cp -r TSUMUGI/js test-tsumugi/
cp -r TSUMUGI/data test-tsumugi/
cp -r TSUMUGI/image test-tsumugi/
cp -r TSUMUGI/network/css test-tsumugi/network/
cp -r TSUMUGI/network/data test-tsumugi/network

In [24]:
%%bash

########################################
# Phenotype
########################################

find data/mp_term_name -type f |
    sed "s|data/mp_term_name/||" |
    sed "s|.csv$||" |
    grep -e ^male_infertility -e ^increased_fasting -e "^prewe" | # <- ここで興味のあるphenotypeを選択
    while read -r mp_term_name_underscore; do
        echo $mp_term_name_underscore

        mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
        impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/annotation/mptermname_phenotypeurl.tsv)
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # --------------------------
        # HTML
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのフィルターを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/network/template/template_network_phenotype.html |
            sed '/<!-- REMOVE FROM THIS LINE IF BINARY -->/,/<!-- REMOVE TO THIS LINE -->/d' |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/network/phenotype/"$mp_term_name_underscore".html
        else
            cat TSUMUGI/network/template/template_network_phenotype.html |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/network/phenotype/"$mp_term_name_underscore".html
        fi

        # --------------------------
        # Javascript
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのスライダーを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/network/template/template_network_phenotype.js |
            sed '/<!-- REMOVE FROM THIS LINE IF BINARY -->/,/<!-- REMOVE TO THIS LINE -->/d' |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/network/phenotype/js/"$mp_term_name_underscore".js
        else
            cat TSUMUGI/network/template/template_network_phenotype.js |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/network/phenotype/js/"$mp_term_name_underscore".js
        fi

        # --------------------------
        # Data
        # --------------------------

        cp TSUMUGI/network/phenotype/data/"$mp_term_name_underscore".json.gz test-tsumugi/network/phenotype/data/
    done

increased_fasting_circulating_glucose_level
male_infertility
preweaning_lethality,_complete_penetrance
preweaning_lethality,_incomplete_penetrance


In [25]:
%%bash

########################################
# Gene Symbol
########################################

cat data/overlap/available_gene_symbols.txt |
    grep Rab10 | # <- ここで興味のあるgene symbolを選択
    while read gene_symbol; do
        echo $gene_symbol

        mgi_id=$(awk -F "\t" -v key="MGI:1918912" '$1 == key {print $2}' data/annotation/symbol_mgiid.tsv)
        impc_url="https://www.mousephenotype.org/data/genes/"${mgi_id}
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # HTML
        cat TSUMUGI/network/template/template_network_genesymbol.html |
        grep -v "REMOVE_THIS_LINE" |
        sed "s|XXX_impc_url|${escaped_url}|g" |
        sed "s|XXX_genesymbol|${gene_symbol}|g" > test-tsumugi/network/genesymbol/"$gene_symbol".html

        # Javascript
        cat TSUMUGI/network/template/template_network_genesymbol.js |
        grep -v "REMOVE_THIS_LINE" |
        sed "s|XXX_genesymbol|${gene_symbol}|g" > test-tsumugi/network/genesymbol/js/"$gene_symbol".js

        # Data
        cp TSUMUGI/network/genesymbol/data/"$gene_symbol".json.gz test-tsumugi/network/genesymbol/data/

    done

Rab10


In [26]:
%%bash

date +"%Y/%m/%d %H:%M:%S" # Last update

2025/02/22 12:17:48
